# Step 3: Calculate OD

In [1]:
import os, sys, time, importlib
import osmnx

import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
sys.path.append("../../../GOSTnets")
import GOSTnets as gn

from shapely.geometry import LineString, Point

In [2]:
# This is a Jupyter Notebook extension which reloads all of the modules whenever you run the code
# This is optional but good if you are modifying and testing source code
%load_ext autoreload
%autoreload 2

In [3]:
from GOSTnets.load_traffic2 import *

In [4]:
# read graph
G = nx.read_gpickle('./sri_lanka_unclean2_w_time_largest_20200616.pickle')

In [5]:
#len(G.edges)

In [6]:
#gn.example_edge(G, 5)

## load origins

In [7]:
origins = gpd.read_file('./origins_destinations/intl_airport.shp')

In [8]:
origins

,osm_id,emergencyh,aeroway,operatorty,source,capacitype,addrfull,emergency,name,addrcity,building,POINT_X,POINT_Y,geometry
0,4.197465e+09,None,helipad,None,None,None,None,None,Ratmalana Int'l Airport,None,None,79.885533,6.823861,POINT (79.88553 6.82386)
1,1.912607e+08,None,None,None,None,None,None,None,Mattala Rajapaksa Int'l Airport,None,None,0.000000,0.000000,POINT (81.12338 6.28615)
2,7.479128e+08,None,None,None,None,None,None,None,Jaffna Int'l Airport,None,None,0.000000,0.000000,POINT (80.06853 9.79439)


## load destinations

In [9]:
destinations = gpd.read_file('./origins_destinations/cities_top10.shp')

In [10]:
destinations

,osm_id,code,fclass,population,name,country,geometry
0,44240758,1001,city,27500,Nuwara Eliya,LKA,POINT (80.76707 6.97382)
1,50794342,1005,national_capital,753000,Colombo,LKA,POINT (79.85385 6.93500)
2,258049423,1001,city,40800,Matale,LKA,POINT (80.62342 7.47213)
3,330173100,1001,city,45400,Puttalam,LKA,POINT (79.82866 8.03019)
4,335253618,1001,city,99100,Trincomalee,LKA,POINT (81.23450 8.57643)
5,2050947471,1001,city,600000,Jaffna,LKA,POINT (80.00930 9.66509)
6,2908772560,1001,city,125000,Kandy,LKA,POINT (80.63508 7.29309)
7,3289447451,1001,city,20000,Gampaha,LKA,POINT (79.99514 7.09256)
8,3639539242,1001,city,110000,Matara,LKA,POINT (80.54829 5.94782)
9,3639546887,1001,city,120000,Galle,LKA,POINT (80.21495 6.03281)


In [11]:
origins_gdf = gn.pandana_snap_c(G, origins, source_crs = 'epsg:4326', target_crs = 'epsg:32644')

In [12]:
#origins_gdf

In [13]:
origins_list = list(set(origins_gdf.NN))

In [14]:
destinations_gdf = gn.pandana_snap_c(G, destinations, source_crs = 'epsg:4326', target_crs = 'epsg:32644')

In [15]:
#destinations_gdf

In [16]:
destinations_list = list(set(destinations_gdf.NN))

In [17]:
#destinations_list

## Calculate OD

In [18]:
# It will use the default weight of 'time'
import time

start = time.time()

OD = gn.calculate_OD(G, origins_list, destinations_list, fail_value = 9999999)

end = time.time()
print(end - start)

17.805907249450684


In [19]:
#OD

In [20]:
OD_df = pd.DataFrame(OD, index = origins_list, columns = destinations_list)

In [21]:
OD_df

,4140985760,4784044133,1734550181,4004620073,3379679501,3289453904,1243386867,7098642100,1727494683,5385270047
4075647849,17291.099786,29138.454346,31511.211558,21981.075558,30537.733749,24456.896561,15061.911802,2382.802628,19462.671844,26253.494821
5456722972,11367.783570,16546.208975,6981.143914,11806.617497,6007.666104,3910.809641,20566.166945,27540.412774,13367.846484,1947.404973
3261840927,18551.875464,12005.816060,3968.955218,17982.673229,5347.630255,10418.162014,25295.903847,34047.765147,19203.026497,10578.605273


In [22]:
#OD_df.min(axis=0)

### Now we need to find the nearest city for each tourist point, and based on this assign a group of tourist points to each of the 10 cities

In [23]:
# test
# first column is the destination (tourist point), and the second column is the nearest city
OD_df.idxmin(axis=0)

4140985760    5456722972
4784044133    3261840927
1734550181    3261840927
4004620073    5456722972
3379679501    3261840927
3289453904    5456722972
1243386867    4075647849
7098642100    4075647849
1727494683    5456722972
5385270047    5456722972
dtype: int64

In [24]:
#OD_df.idxmin(axis=0).to_csv('./nodes_and_associated_nearest_city.csv')

In [25]:
#OD_df.idxmin(axis=0).to_frame(0)

### takes the min index value of each column, then groups by city (first index (0)) and takes the first entry

In [26]:
groupby_obj = OD_df.idxmin(axis=0).to_frame(0).groupby(0)[0]

In [28]:
groupby_obj

In [29]:
# visualize groupby_obj
#groupby_obj.apply(list)

In [30]:
#type(groupby_obj.apply(list))

In [31]:
# a nice way to visualize the groupby_obj
#groupby_obj.describe()

### create a dictionary that associates assigned tourist points with each city

In [32]:
city_tourist_pt_dict = {}
for name, group in groupby_obj:
    #print(group)
    for items in group.iteritems(): 
        #print(items[1])
        if items[1] not in city_tourist_pt_dict:
            city_tourist_pt_dict[items[1]] = [items[0]]
        else:
            #append value to list in dict value
            city_tourist_pt_dict[items[1]].append(items[0])
    #print(type(group))
    #print(group.head(1))

    #print(name)
    #print(city_tourist_pt_dict[group])

In [33]:
city_tourist_pt_dict

{3261840927: [4784044133, 1734550181, 3379679501],
 4075647849: [1243386867, 7098642100],
 5456722972: [4140985760, 4004620073, 3289453904, 1727494683, 5385270047]}

## Loop through dictionary in order to do a calculate_OD for each airport's nearest cities

In [34]:
OD = {}
OD_df_dict = {}
for city,tourist_pts in city_tourist_pt_dict.items():
    OD[city] = gn.calculate_OD(G, [city], tourist_pts, fail_value = 9999999)
    OD_df_dict[city] = pd.DataFrame(OD[city], index = [city], columns = tourist_pts)

In [35]:
#sample_df = OD_df_dict[4784044133]
#sample_df

In [36]:
OD_df_dict

{3261840927:              4784044133   1734550181   3379679501
 3261840927  12005.81606  3968.955218  5347.630255,
 4075647849:               1243386867   7098642100
 4075647849  15061.911802  2382.802628,
 5456722972:              4140985760    4004620073   3289453904    1727494683   5385270047
 5456722972  11367.78357  11806.617497  3910.809641  13367.846484  1947.404973}

### time calculateOD time for airport city and its set of city points

In [37]:
#start = time.time()
#OD[4784044133] = gn.calculate_OD(G, [4784044133], tourist_pts, fail_value = 9999999)
#print(time.time() - start)

In [38]:
# sum of all shortest routes
#OD[4784044133][0].sum()

## Now work on generating routes and visualizing them

In [39]:
from shapely.ops import linemerge
from itertools import islice

In [40]:
maxspeed_mean_speeds = {
    'secondary': 50,
    'secondary_link': 45,
    'tertiary': 40,
    'tertiary_link': 40,
    'residential': 25,
    'unclassified': 25,
}

In [41]:
def tabulate_edges(route):
    edge_table = []
    route_geometry = LineString()
    for idx in range(0, len(route) - 1):
        #edge_table.append([route[idx], route[idx+1]])
        # look up line
        #print('to node')
        #print(route[idx])
        #print('from node')
        #print(route[idx+1])
        edge_geometry = G.get_edge_data(route[idx],route[idx+1])[0]['geometry']
        # get edge speed
        edge_speed = G.get_edge_data(route[idx],route[idx+1])[0]['speed']
        #print('print edge_speed')
        #print(edge_speed)
        
        # compare edge speed to median speed
        rural_roads_list = ['residential','secondary','secondary_link','tertiary','tertiary_link','unclassified']
        
        edge_infra_type = G.get_edge_data(route[idx],route[idx+1])[0]['infra_type']
        
        edge_length = G.get_edge_data(route[idx],route[idx+1])[0]['length']
        edge_time = G.get_edge_data(route[idx],route[idx+1])[0]['time']
        
        mean_speed = G.get_edge_data(route[idx],route[idx+1])[0]['mean_speed']
        
        try:
            edge_imp_cost = G.get_edge_data(route[idx],route[idx+1])[0]['imp_cost']
        except:
            edge_imp_cost = 0
            pass
        
        if mean_speed > 0:      
            if edge_infra_type in rural_roads_list:
                print('print edge attributes')
                print(G.get_edge_data(route[idx],route[idx+1])[0])
                #print('print improved speed')
                #print(mapbox_traffic_mean_speeds[edge_infra_type])
                #assums that current edge lenght is in km
                new_time_s = (edge_length / maxspeed_mean_speeds[edge_infra_type]) * 3600
                #new_time_s = ((edge_length / 1000) / maxspeed_mean_speeds[edge_infra_type]) * 3600
                #print("print new_time_s")
                #print(new_time_s)
                #print('print old time')
                #print(edge_time)
                edge_savings = edge_time - new_time_s
                # assign savings time
                edge_table.append([route[idx], route[idx+1], edge_savings, edge_imp_cost, edge_length, edge_infra_type, mean_speed, edge_geometry])
                # data['sec_saved'] = data['length'] / new_time
                # data['improvement_cost'] = 174861 * data['length'] / 1000
            else:
                # data['sec_saved'] = 0
                # very high number
                # data['improvement_cost'] = 10000000000
                #print("not a rural road")
                edge_table.append([route[idx], route[idx+1], 0, 0, edge_length, edge_infra_type, mean_speed, edge_geometry])
        else:
            edge_table.append([route[idx], route[idx+1], 0, 0, edge_length, edge_infra_type, mean_speed, edge_geometry])
            
        route_geometry = route_geometry.union(edge_geometry)
        
    #print('print route_geometry')
    #print(route_geometry)
    
    return(edge_table, route_geometry)

In [42]:
def generate_complete_edges_and_routes(input_df):

    LIMIT = 1000000

    complete_edges = []
    complete_routes = []

    count = 0

    # for origin, row in sample_df.iterrows(): 
    for origin, row in islice(input_df.iterrows(), LIMIT):    
        for destination, value in islice(row.items(), LIMIT):
            try:
                origin = int(origin)
            except:
                pass
            try:
                destination = int(destination)
            except:
                pass

            count = count + 1

            route = nx.dijkstra_path(G, origin, destination, weight = 'time')
            #path_edges = zip(route,route[1:])
            #print('print path_edges')
            #print(list(path_edges))
            # print(route)
            edge_table, route_geometry = tabulate_edges(route)
            #print('print edge_table:')
            #print(edge_table)
            complete_edges = complete_edges + edge_table
            #print('route_time')
            #print(value)
            complete_routes.append([edge_table[0][0], edge_table[-1][1], value, route_geometry])
            #print('edge_table[:-1]')
            #print(edge_table[-1][1])
            
    # convert complete_edges to gdf
    complete = pd.DataFrame(complete_edges, columns = ['o', 'd', 'sec_saved', 'imp_cost', 'length', 'infra_type', 'mean_speed', 'geometry'])
    complete['w'] = 1
    complete_count = complete.groupby(['o','d']).agg(
        {
            'w':"count",
            'sec_saved': 'first',
            'imp_cost': 'first',
            'mean_speed': 'first',
            'length':'first',
            'infra_type':'first',
            'geometry':'first'
        }
    )
    complete_count.reset_index(inplace = True)
    complete_count['o'] = complete_count['o'].astype(str)
    complete_count['d'] = complete_count['d'].astype(str)
    complete_count['weighted_sec_saved'] = complete_count.w * complete_count.sec_saved
    complete_count.sort_values(by=['weighted_sec_saved'], ascending=False)
    complete_count_gdf = gpd.GeoDataFrame(complete_count, crs = 'epsg:4326')
    
    # convert complete_routes to gdf
    complete_routes_df = pd.DataFrame(complete_routes, columns = ['origin','destination','time','geometry'])
    complete_routes_gdf = gpd.GeoDataFrame(complete_routes_df, crs = 'epsg:4326')
        
    return [complete_count_gdf, complete_routes_gdf]

In [43]:
#results = generate_complete_edges_and_routes(OD_df[4784044133])

In [44]:
OD_df_dict

{3261840927:              4784044133   1734550181   3379679501
 3261840927  12005.81606  3968.955218  5347.630255,
 4075647849:               1243386867   7098642100
 4075647849  15061.911802  2382.802628,
 5456722972:              4140985760    4004620073   3289453904    1727494683   5385270047
 5456722972  11367.78357  11806.617497  3910.809641  13367.846484  1947.404973}

In [45]:
for key in OD_df_dict:
    print(key)

3261840927
4075647849
5456722972


In [46]:
import time

start = time.time()

results = {}

count = 0
for key in OD_df_dict:
    #while count > 1 and count < 6:
    results[key] = generate_complete_edges_and_routes(OD_df_dict[key])
    #count += 1
    
print(time.time() - start)

print edge attributes
{'osm_id': 513498108, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb58a3bae10>, 'infra_type': 'tertiary', 'min_speed': 29.0, 'max_speed': 29.0, 'mean_speed': 29.0, 'length': 0.016792262703916958, 'speed': 29.0, 'imp_cost': 3136.794673091688, 'time': 2.0845567494517603, 'mode': 'drive'}
print edge attributes
{'osm_id': 513498108, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb56235a550>, 'infra_type': 'tertiary', 'min_speed': 29.0, 'max_speed': 29.0, 'mean_speed': 29.0, 'length': 0.012310334102251946, 'speed': 29.0, 'imp_cost': 2299.5704103006633, 'time': 1.5281794057967932, 'mode': 'drive'}
print edge attributes
{'osm_id': 513498108, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb56235a4d0>, 'infra_type': 'tertiary', 'min_speed': 29.0, 'max_speed': 29.0, 'mean_speed': 29.0, 'length': 0.01661310133916969, 'speed': 29.0, 'imp_cost': 3103.3273301568984, 'time': 2.06231602831072, 'mode': 'drive'}
print edge 

print edge attributes
{'osm_id': 513498108, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb56235ab10>, 'infra_type': 'tertiary', 'min_speed': 29.0, 'max_speed': 29.0, 'mean_speed': 29.0, 'length': 0.07795389590211159, 'speed': 29.0, 'imp_cost': 14561.787754514446, 'time': 9.677035353365577, 'mode': 'drive'}
print edge attributes
{'osm_id': 513498108, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb56235ab50>, 'infra_type': 'tertiary', 'min_speed': 29.0, 'max_speed': 29.0, 'mean_speed': 29.0, 'length': 0.09788322459781133, 'speed': 29.0, 'imp_cost': 18284.586354871157, 'time': 12.15102098455589, 'mode': 'drive'}
print edge attributes
{'osm_id': 513498108, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb56235ab90>, 'infra_type': 'tertiary', 'min_speed': 29.0, 'max_speed': 29.0, 'mean_speed': 29.0, 'length': 0.0353080699599192, 'speed': 29.0, 'imp_cost': 6595.547468512906, 'time': 4.383070753645141, 'mode': 'drive'}
print edge attr

print edge attributes
{'osm_id': 513498108, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb5622f8410>, 'infra_type': 'tertiary', 'min_speed': 29.0, 'max_speed': 29.0, 'mean_speed': 29.0, 'length': 0.018812303697267396, 'speed': 29.0, 'imp_cost': 3514.1383306495495, 'time': 2.3353204589711254, 'mode': 'drive'}
print edge attributes
{'osm_id': 513498108, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb5622f8450>, 'infra_type': 'tertiary', 'min_speed': 29.0, 'max_speed': 29.0, 'mean_speed': 29.0, 'length': 0.017238994852069058, 'speed': 29.0, 'imp_cost': 3220.2442383665, 'time': 2.1400131540499516, 'mode': 'drive'}
print edge attributes
{'osm_id': 513498108, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb5622f8490>, 'infra_type': 'tertiary', 'min_speed': 29.0, 'max_speed': 29.0, 'mean_speed': 29.0, 'length': 0.015355249636350272, 'speed': 29.0, 'imp_cost': 2868.360632070231, 'time': 1.9061689203745167, 'mode': 'drive'}
print edge 

print edge attributes
{'osm_id': 513498108, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb5622f8c90>, 'infra_type': 'tertiary', 'min_speed': 29.0, 'max_speed': 29.0, 'mean_speed': 29.0, 'length': 0.010514377822988422, 'speed': 29.0, 'imp_cost': 1964.0857773342373, 'time': 1.3052331090606317, 'mode': 'drive'}
print edge attributes
{'osm_id': 513498108, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb5622f8cd0>, 'infra_type': 'tertiary', 'min_speed': 29.0, 'max_speed': 29.0, 'mean_speed': 29.0, 'length': 0.012260223711723778, 'speed': 29.0, 'imp_cost': 2290.2097893500018, 'time': 1.5219588055932967, 'mode': 'drive'}
print edge attributes
{'osm_id': 513498108, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb5622f8d10>, 'infra_type': 'tertiary', 'min_speed': 29.0, 'max_speed': 29.0, 'mean_speed': 29.0, 'length': 0.013840241015968426, 'speed': 29.0, 'imp_cost': 2585.357021782902, 'time': 1.718098884740908, 'mode': 'drive'}
print edg

print edge attributes
{'osm_id': 513498108, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb562314490>, 'infra_type': 'tertiary', 'min_speed': 29.0, 'max_speed': 29.0, 'mean_speed': 29.0, 'length': 0.003606412187268561, 'speed': 29.0, 'imp_cost': 673.6777965817672, 'time': 0.44769254738506276, 'mode': 'drive'}
print edge attributes
{'osm_id': 513498108, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb5623144d0>, 'infra_type': 'tertiary', 'min_speed': 29.0, 'max_speed': 29.0, 'mean_speed': 29.0, 'length': 0.0032948020821604576, 'speed': 29.0, 'imp_cost': 615.4690289475735, 'time': 0.4090099136475051, 'mode': 'drive'}
print edge attributes
{'osm_id': 513498108, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb562314510>, 'infra_type': 'tertiary', 'min_speed': 29.0, 'max_speed': 29.0, 'mean_speed': 29.0, 'length': 0.004887368825250049, 'speed': 29.0, 'imp_cost': 912.9604965567092, 'time': 0.6067078541689717, 'mode': 'drive'}
print ed

...

print edge attributes
{'osm_id': 371237411, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb57bafa890>, 'infra_type': 'secondary', 'min_speed': 41.0, 'max_speed': 41.0, 'mean_speed': 41.0, 'length': 0.06084850417615836, 'speed': 41.0, 'imp_cost': 11366.500580106382, 'time': 5.342795488638295, 'mode': 'drive'}
print edge attributes
{'osm_id': 371237411, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb57bafa850>, 'infra_type': 'secondary', 'min_speed': 41.0, 'max_speed': 41.0, 'mean_speed': 41.0, 'length': 0.05894042896639249, 'speed': 41.0, 'imp_cost': 11010.072130922117, 'time': 5.175257177536902, 'mode': 'drive'}
print edge attributes
{'osm_id': 371237411, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb57bafa810>, 'infra_type': 'secondary', 'min_speed': 41.0, 'max_speed': 41.0, 'mean_speed': 41.0, 'length': 0.08962653809862421, 'speed': 41.0, 'imp_cost': 16742.237316823004, 'time': 7.869647247684078, 'mode': 'drive'}
print edge

print edge attributes
{'osm_id': 371237411, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb57bafa050>, 'infra_type': 'secondary', 'min_speed': 29.0, 'max_speed': 29.0, 'mean_speed': 29.0, 'length': 0.0729364637990903, 'speed': 29.0, 'imp_cost': 13624.531437670068, 'time': 9.05418171299052, 'mode': 'drive'}
print edge attributes
{'osm_id': 371237411, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb57bafa090>, 'infra_type': 'secondary', 'min_speed': 29.0, 'max_speed': 29.0, 'mean_speed': 29.0, 'length': 0.04442463442785745, 'speed': 29.0, 'imp_cost': 8298.52171112377, 'time': 5.514782204837476, 'mode': 'drive'}
print edge attributes
{'osm_id': 371237411, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb57bb5af90>, 'infra_type': 'secondary', 'min_speed': 29.0, 'max_speed': 29.0, 'mean_speed': 29.0, 'length': 0.02835607821302691, 'speed': 29.0, 'imp_cost': 5296.915410193426, 'time': 3.5200648816171336, 'mode': 'drive'}
print edge att

print edge attributes
{'osm_id': 23368458, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb5b14dc210>, 'infra_type': 'tertiary', 'min_speed': 9.0, 'max_speed': 41.0, 'mean_speed': 28.53472222222222, 'length': 0.039388392720196196, 'speed': 28.53472222222222, 'imp_cost': 7357.751760132649, 'time': 4.969321680737335, 'mode': 'drive'}
print edge attributes
{'osm_id': 23368458, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb5b14dc2d0>, 'infra_type': 'tertiary', 'min_speed': 15.0, 'max_speed': 35.0, 'mean_speed': 26.483630952380953, 'length': 0.053384928591305836, 'speed': 26.483630952380953, 'imp_cost': 9972.30466085593, 'time': 7.256774695065858, 'mode': 'drive'}
print edge attributes
{'osm_id': 378852894, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb5adca2f50>, 'infra_type': 'tertiary', 'min_speed': 24.0, 'max_speed': 30.0, 'mean_speed': 24.223214285714285, 'length': 0.0677318248650765, 'speed': 24.223214285714285, 'imp_cost': 

print edge attributes
{'osm_id': 378848515, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb5b1c79310>, 'infra_type': 'tertiary', 'min_speed': 11.0, 'max_speed': 50.0, 'mean_speed': 33.23412698412698, 'length': 0.0563598626707142, 'speed': 33.23412698412698, 'imp_cost': 10528.022346889413, 'time': 6.105034915208589, 'mode': 'drive'}
print edge attributes
{'osm_id': 378848515, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb5ac94c310>, 'infra_type': 'tertiary', 'min_speed': 12.0, 'max_speed': 58.0, 'mean_speed': 40.23561507936508, 'length': 0.10352545731114096, 'speed': 40.23561507936508, 'imp_cost': 19338.555425721133, 'time': 9.262730185308962, 'mode': 'drive'}
print edge attributes
{'osm_id': 378848515, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fb5ac94c250>, 'infra_type': 'tertiary', 'min_speed': 6.0, 'max_speed': 58.0, 'mean_speed': 42.41468253968254, 'length': 0.042944085602466806, 'speed': 42.41468253968254, 'imp_cost': 

972.4654824733734


In [47]:
for key in results:
    # print edges
    #print(results[key][0])
    file_name = f"./airport_output_edges_20200628/weighted_sec_saved_edges_{key}.shp"
    print(file_name)
    results[key][0].to_file(driver = 'ESRI Shapefile', filename = file_name)

./airport_output_edges_20200628/weighted_sec_saved_edges_3261840927.shp
./airport_output_edges_20200628/weighted_sec_saved_edges_4075647849.shp
./airport_output_edges_20200628/weighted_sec_saved_edges_5456722972.shp


In [48]:
#results[1243386867][0]

In [49]:
# save as shapefile
#results[0].to_file(driver = 'ESRI Shapefile', filename = "./weighted_sec_saved_edges.shp")

In [50]:
# save as shapefile
#results[1].to_file(driver = 'ESRI Shapefile', filename = "./shortest_path_routes.shp")